# Asset Class Trend Following 策略回測與報告 (etf2023)

In [1]:
# --- 全域參數設定區塊 ---
SMA_PERIOD = 64
ROC_PERIOD = 23
STOP_LOSS_PCT = 0.09
INITIAL_CAPITAL = 30000000
DATA_FILE = '154ETF-2.xlsx'
OUTPUT_EXCEL = 'trendstrategy_results_etf2023.xlsx'
OUTPUT_MD = 'reproduce_report_etf2023.md'
# -----------------------

import pandas as pd
import numpy as np
import os


In [2]:
def clean_data(filepath):
    df_raw = pd.read_excel(filepath, header=None)
    # Row 0: NaN, 股票代號, 0050, ...
    stock_codes = df_raw.iloc[0, 2:].values
    # Row 1: 日期, 股票名稱, 元大台灣50, ...
    stock_names = df_raw.iloc[1, 2:].values
    # Col 0: 20220901, ...
    dates = pd.to_datetime(df_raw.iloc[2:, 0], format='%Y%m%d')
    prices = df_raw.iloc[2:, 2:].astype(float)
    prices.index = dates
    prices.columns = stock_codes
    code_to_name = dict(zip(stock_codes, stock_names))
    
    # 規則: 若資料起始日起未有資料者，就以第一筆出現的收盤價往前填補 (bfill)
    # 規則: 若遇到交易日中間有空白值，就以前一交易日收盤價填補中間空白值 (ffill)
    prices = prices.ffill().bfill()
    
    prices = prices.dropna(axis=1, how='all')
    return prices, code_to_name


In [3]:
class Backtester:
    def __init__(self, prices, code_to_name, initial_capital=30000000):
        self.prices = prices.values
        self.dates = prices.index
        self.assets = prices.columns
        self.code_to_name = code_to_name
        self.initial_capital = initial_capital
        
    def run(self, sma_period, roc_period, stop_loss_pct):
        prices_df = pd.DataFrame(self.prices, index=self.dates, columns=self.assets)
        sma = prices_df.rolling(window=sma_period).mean().values
        roc = prices_df.pct_change(periods=roc_period).values
        
        capital = self.initial_capital
        portfolio = {} 
        equity_curve = np.zeros(len(self.dates))
        rebalance_log = []
        holdings_history = []
        
        start_idx = max(sma_period, roc_period)
        
        for i in range(start_idx, len(self.dates) - 1):
            date = self.dates[i]
            current_prices = self.prices[i]
            next_prices = self.prices[i+1]
            
            total_equity = capital
            assets_to_sell = []
            
            for asset_idx, info in list(portfolio.items()):
                curr_p = current_prices[asset_idx]
                total_equity += info['shares'] * curr_p
                if curr_p > info['max_price']:
                    info['max_price'] = curr_p
                if curr_p < info['max_price'] * (1 - stop_loss_pct):
                    assets_to_sell.append(asset_idx)
            
            equity_curve[i] = total_equity
            is_rebalance_day = (i - start_idx) % 5 == 0
            
            new_portfolio_signals = []
            if is_rebalance_day:
                eligible_mask = (current_prices > sma[i]) & (roc[i] > 0)
                if np.any(eligible_mask):
                    eligible_idxs = np.where(eligible_mask)[0]
                    eligible_rocs = roc[i][eligible_idxs]
                    top_k = min(3, len(eligible_idxs))
                    top_idxs = eligible_idxs[np.argsort(eligible_rocs)[-top_k:][::-1]]
                    new_portfolio_signals = list(top_idxs)
            
            assets_selling_now = set(assets_to_sell)
            if is_rebalance_day:
                for asset_idx in list(portfolio.keys()):
                    if asset_idx not in new_portfolio_signals:
                        assets_selling_now.add(asset_idx)
            
            # Log non-rebalance stop losses
            if not is_rebalance_day:
                for asset_idx in assets_to_sell:
                    if asset_idx in portfolio:
                        rebalance_log.append({
                            '日期': date, '股票代號': self.assets[asset_idx], '狀態': "賣出",
                            '價格': current_prices[asset_idx], '股數': 0,
                            '動能值': f"{roc[i][asset_idx]*100:.2f}%",
                            '標的名稱': self.code_to_name.get(self.assets[asset_idx], ""),
                            '最佳參數': f"SMA={sma_period}, ROC={roc_period}, SL={stop_loss_pct}",
                            '原因': "停損",
                            '說明': f"觸發停損：{self.code_to_name.get(self.assets[asset_idx], '')} ({self.assets[asset_idx]})"
                        })

            for asset_idx in assets_selling_now:
                if asset_idx in portfolio:
                    info = portfolio.pop(asset_idx)
                    sell_price = next_prices[asset_idx]
                    capital += info['shares'] * sell_price
            
            if is_rebalance_day:
                assets_to_buy = [a for a in new_portfolio_signals if a not in portfolio]
                slot_capital = self.initial_capital / 3
                for asset_idx in assets_to_buy:
                    buy_price = next_prices[asset_idx]
                    shares = slot_capital // buy_price
                    if shares > 0 and capital >= shares * buy_price:
                        capital -= shares * buy_price
                        portfolio[asset_idx] = {
                            'shares': shares, 'buy_price': buy_price, 'buy_date': self.dates[i+1],
                            'max_price': buy_price, 'momentum': roc[i][asset_idx]
                        }
                
                # Log on rebalance day
                for asset_idx in range(len(self.assets)):
                    status, reason = "", ""
                    if asset_idx in portfolio:
                        if asset_idx in assets_to_buy: status, reason = "買進", "符合趨勢"
                        else: status, reason = "保持", "趨勢持續"
                    elif asset_idx in assets_selling_now:
                        status = "賣出"
                        reason = "停損" if asset_idx in assets_to_sell else "再平衡"
                    
                    if status:
                        rebalance_log.append({
                            '日期': date, '股票代號': self.assets[asset_idx], '狀態': status,
                            '價格': current_prices[asset_idx],
                            '股數': portfolio[asset_idx]['shares'] if asset_idx in portfolio else 0,
                            '動能值': f"{roc[i][asset_idx]*100:.2f}%",
                            '標的名稱': self.code_to_name.get(self.assets[asset_idx], ""),
                            '最佳參數': f"SMA={sma_period}, ROC={roc_period}, SL={stop_loss_pct}",
                            '原因': reason,
                            '說明': f"選取資產：{self.code_to_name.get(self.assets[asset_idx], '')} ({self.assets[asset_idx]})，動能：{roc[i][asset_idx]*100:.2f}%"
                        })

            holdings_history.append({
                'Date': date,
                'Holdings': ", ".join([str(self.assets[a]) for a in portfolio.keys()]),
                'Equity': total_equity
            })

        equity_curve[-1] = total_equity
        eq_series = pd.Series(equity_curve, index=self.dates).replace(0, np.nan).dropna()
        return eq_series, pd.DataFrame(rebalance_log), pd.DataFrame(holdings_history)


In [4]:
def calculate_metrics(equity_curve):
    if equity_curve.empty: return 0, 0, 0
    total_return = (equity_curve.iloc[-1] / equity_curve.iloc[0]) - 1
    days = (equity_curve.index[-1] - equity_curve.index[0]).days
    if days == 0: return 0, 0, 0
    cagr = (1 + total_return) ** (365.25 / days) - 1
    rolling_max = equity_curve.cummax()
    drawdown = (equity_curve - rolling_max) / rolling_max
    max_dd = drawdown.min()
    calmar = cagr / abs(max_dd) if max_dd != 0 else 0
    return cagr, max_dd, calmar


In [5]:
prices, code_to_name = clean_data(DATA_FILE)
bt = Backtester(prices, code_to_name, INITIAL_CAPITAL)
eq, trades_df, holdings_df = bt.run(SMA_PERIOD, ROC_PERIOD, STOP_LOSS_PCT)
cagr, mdd, calmar = calculate_metrics(eq)

print(f"SMA={SMA_PERIOD}, ROC={ROC_PERIOD}, SL={STOP_LOSS_PCT}")
print(f"CAGR: {cagr:.2%}, MaxDD: {mdd:.2%}, Calmar: {calmar:.2f}")

with pd.ExcelWriter(OUTPUT_EXCEL, engine='xlsxwriter') as writer:
    trades_df.to_excel(writer, sheet_name='Trades', index=False)
    equity_curve_df = pd.DataFrame({'Date': eq.index, 'Equity': eq.values})
    equity_curve_df.to_excel(writer, sheet_name='Equity_Curve', index=False)
    holdings_df.to_excel(writer, sheet_name='Equity_Hold', index=False)
    summary_df = pd.DataFrame([
        {'Metric': 'CAGR', 'Value': f"{cagr:.2%}"},
        {'Metric': 'MaxDD', 'Value': f"{mdd:.2%}"},
        {'Metric': 'Calmar Ratio', 'Value': f"{calmar:.2f}"},
        {'Metric': 'SMA', 'Value': SMA_PERIOD},
        {'Metric': 'ROC', 'Value': ROC_PERIOD},
        {'Metric': 'StopLoss%', 'Value': STOP_LOSS_PCT}
    ])
    summary_df.to_excel(writer, sheet_name='Summary', index=False)

print(f"結果已儲存至 {OUTPUT_EXCEL}")


SMA=64, ROC=23, SL=0.09
CAGR: 21.12%, MaxDD: -26.41%, Calmar: 0.80


結果已儲存至 trendstrategy_results_etf2023.xlsx
